Read Keypoint Labels

In [1]:
import pandas as pd
labels = pd.read_pickle("./dataset/labels.pkl")

In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


Write txt files for torch training. Downloaded data is in ./dataset/cropped_images 

In [4]:
from os import mkdir
import shutil
#split from paper
d1_val = ['d1_02_06_2020', 'd1_02_16_2020', 'd1_02_22_2020']
d1_test = ['d1_03_03_2020', 'd1_03_19_2020', 'd1_03_23_2020', 'd1_03_27_2020', 'd1_03_28_2020', 'd1_03_30_2020', 'd1_03_31_2020']

d2_val = ['d2_02_03_2021', 'd2_02_05_2021']
d2_test = ['d2_03_03_2020', 'd2_02_10_2021', 'd2_02_03_2021_2']
#new split:
val = d1_val + d2_val
test = d1_test + d2_test
mkdir("./dataset/train")
mkdir("./dataset/val")
mkdir("./dataset/test")
for index, row in labels.iterrows():
    typ = "train"
    if row["img_folder"] in val:
        typ = "val"
    elif row["img_folder"] in test:
        typ = "test"
    img_path = "./dataset/cropped_images/800/{}/{}".format(row["img_folder"],row["img_name"])
    dest = "./dataset/{}/{}".format(typ,row["img_name"])
    shutil.copyfile(img_path,dest)
    txt = open("./dataset/{}/{}.txt".format(typ,row["img_name"].split(".")[0]),"w")
    #skip first 3 labels and only one class
    for xy in row["xy"][4:]:
        txt.write("0 {} {} 0.075 0.075\n".format(xy[0],xy[1]))
    txt.close()


In [4]:
import torch
import yaml
from IPython.display import Image, clear_output
from utils.plots import plot_results
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.11.0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070', major=8, minor=6, total_memory=8191MB, multi_processor_count=46)


In [ ]:
# PyTorch Hub
import torch
from torchvision.io.image import read_image
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image


# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.eval()
# Images
dir = 'dataset/cropped_images/800'
imgs = [dir + f for f in ('zidane.jpg', 'bus.jpg')]  # batch of images

# Inference
results = model(imgs)
results.show()  # or .show(), .save()

In [ ]:

img = read_image("test/assets/encode_jpeg/grace_hopper_517x606.jpg")

# Step 1: Initialize model with the best available weights
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)


# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = [preprocess(img)]

# Step 4: Use the model and visualize the prediction
prediction = model(batch)[0]
labels = [weights.meta["categories"][i] for i in prediction["labels"]]
box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                          labels=labels,
                          colors="red",
                          width=4, font_size=30)
im = to_pil_image(box.detach())
im.show()